# Build a blocklist from seeds

Build a list of NSFW Wikipedia categories to block starting from a set of seeds.

This notebook requires the category index dataset to have been built previously by running `build_category_index.ipynb`.

In [1]:
from pathlib import Path

import pandas as pd

from wikipedia_utils.utils import display_pd
from wikipedia_utils import category

`DATA_DIR` should be set to the dir where the category index files were written.

In [2]:
DATA_DIR = Path("category_data")
SEED_FILE = "moderation_category_seeds.yml"
NEW_BLOCKLIST_CSV = DATA_DIR / "blocklist_cats.csv"
CURRENT_BLOCKLIST_CSV = "blocklist_cats.csv"

## Load the seed file

The final list of categories to block is built by expanding on a manually curated set of seeds.
The seeds are defined in a YAML file and consist of inclusions and exclusions, either exact category names and regexes, for each topic area.
These seeds have been chosen by interactively exploring Wikipedia categories relevant to the topic areas of interest using the tools described in `explore_categories.ipynb`.

The seed file has the following format. For each topic listing, all keys except for `topic` are optional.
```yaml
blocklist:
  - topic: <topic_key>
    seed_categories:
      - <exact category name>
    seed_re:
      - <regex to match category names>
    ignore_categories:
      - <exact category name>
    ignore_re:
      - <regex to match category names>
    max_level: <max depth to search category digraph>
```

In [3]:
seed_dict = category.load_blocklist_seeds(SEED_FILE)

We can view the seed list as a table with columns for each NSFW topic.

In [4]:
display_pd(pd.DataFrame(seed_dict).drop("topic"))

,profanity,pejoratives,cruelty,child_abuse,sexually_explicit,hateful_ideologies
seed_categories,[Profanity],NaN,[Torture],[Child abuse],"[Sex manuals, Incest, Sexual emotions, Prostitution, Sexuality-related lists, Machine sex, Orgasm, Sexual fetishism, Pornography, Paraphilias, Sexual slang, Personal lubricants, Sexual fantasies, Sex toys, Sexual violence, Sexual acts]",[White nationalism]
ignore_categories,[Works about profanity],[Barbarians],"[Castrated people, Operation Condor, Crucifixion, Amputations, Fictional torturers, Works about torture, Guantanamo Bay detention camp, People of the Dirty War]","[Child abuse law, Child abuse-related organizations, Anti-pedophile activism, Child labour, Displacement of indigenous children, School bullying, QAnon, Works about child abuse, Feral children, Fiction about child murder, Filicides, Fictional murderers of children, Child sacrifice]","[One Thousand and One Nights, Aphrodite, Game of Thrones, Erotic Liquid Culture members, Swedish Erotica members]",[Works about apartheid]
seed_re,NaN,"[\bslurs, \bpejorative]",NaN,NaN,"[\bpornograph, \berotic]",NaN
ignore_re,NaN,[\beskimo],"[\banti-torture\b, \btorture victims?\b, \bhuman rights activ]",NaN,"[\banti\W(child )?porn, \blaw.+\bporn, \bporn.+\blaw, \blaw.+\bprostitut, \bprostitut.+\blaw, \banti\Wprostitut, \bsex comedy\b, \bmytholog, \bNew Pornographers\b]",NaN
max_level,NaN,NaN,NaN,NaN,NaN,4


## Build blocklists for individual topics

To build a list of categories that will be used as a blocklist:

1. Identify all categories matched by the seeds, either exact category names or matching regexes. If seeds match both a subcategory and its parent, keep only the parent to ensure a cleaner level structure.
2. For each of these, identify all their subcategories, ignoring specified exact category names or matching regexes.
3. Repeat step 2 for all newly identified subcategories, until the depth cutoff is reached, or no more subcategories are available.

We can do this for individual topic specs included in the seed file. This is useful for verifying the final list of categories that will be included in the blocklist.

In [5]:
ci = category.CategoryIndex("category_data")

Individual blocklists for each category should be manually inspected to make sure they cover the desired set of topics.

In [6]:
%%time

ci.build_category_list_for_topic(seed_dict["profanity"])

CPU times: user 1.33 s, sys: 168 ms, total: 1.5 s
Wall time: 1.52 s


,name,seed,parent,level,topic
0,Profanity,Profanity,None,0,profanity
1,Profanity by language,Profanity,Profanity,1,profanity
2,Cantonese profanity,Profanity,Profanity by language,2,profanity
3,English profanity,Profanity,Profanity by language,2,profanity
4,Finnish profanity,Profanity,Profanity by language,2,profanity
5,French profanity,Profanity,Profanity by language,2,profanity
6,German profanity,Profanity,Profanity by language,2,profanity
7,Hindi profanity,Profanity,Profanity by language,2,profanity
8,Polish profanity,Profanity,Profanity by language,2,profanity
9,Russian profanity,Profanity,Profanity by language,2,profanity


## Build full blocklist

Finally, we build the full blocklist for the entire seed list.
This is done using the same method as above, across all topics, removing any duplicates.
The result is written directly to a CSV file.

In [7]:
%%time

ci.build_full_category_list(seed_dict, NEW_BLOCKLIST_CSV)

CPU times: user 22.9 s, sys: 2.01 s, total: 24.9 s
Wall time: 25 s


Take a look at the generated blocklist of categories.

In [8]:
blocklist_cats = pd.read_csv(NEW_BLOCKLIST_CSV)

In [9]:
len(blocklist_cats)

3274

Compute some info for each topic:

In [10]:
blocklist_cats.groupby("topic").agg(
    num_categories=pd.NamedAgg("name", "size"),
    max_level=pd.NamedAgg("level", "max")
)

,num_categories,max_level
topic,,
child_abuse,402,8
cruelty,132,7
hateful_ideologies,303,4
pejoratives,32,4
profanity,11,2
sexually_explicit,2394,10


__NSFW content warning!__

Take a look at some of the categories

In [11]:
blocklist_cats.sample(20)

,name,seed,parent,level,topic
1528,Sexual harassment journalism,Sexual violence,Sexual harassment,3,sexually_explicit
2248,Pakistani sex offenders,Sexual violence,Sex offenders by nationality,4,sexually_explicit
2502,Pornography in Nevada,Erotic,Pornography in the United States by state,5,sexually_explicit
3129,Nazi politicians,White nationalism,White supremacists,3,hateful_ideologies
1468,Rape in South Korea,Sexual violence,Rape by country,3,sexually_explicit
1603,German erotic artists,Erotic,Erotic artists by nationality,4,sexually_explicit
2044,Courtesans from Paris,Prostitution,French courtesans,4,sexually_explicit
94,Torture in Spain,Torture,Torture by country,2,cruelty
2035,Films about prostitution in Australia,Prostitution,Works about prostitution in Australia,4,sexually_explicit
690,Burlesque,Erotic,Erotic dance,2,sexually_explicit


## Compare against previous blocklist

When we recompute the blocklist using a new category dump, we will want to see what changed relative to the previous version.

In [12]:
prev_bl_cats = pd.read_csv(CURRENT_BLOCKLIST_CSV)
new_bl_cats = pd.read_csv(NEW_BLOCKLIST_CSV)

In [13]:
###-------------------------------
### DELETEME
### This is only needed temporarily to convert the initial list to the new format.

prev_seeds = prev_bl_cats["seed"].unique()

prev_seed_topic = {}
for s in prev_seeds:
    for v in seed_dict.values():
        s_cats = [c.lower() for c in v.get("seed_categories", [])]
        s_re = [r.replace("\\b", "") for r in v.get("seed_re", [])]
        if s.lower() in s_cats or s.lower() in s_re:
            prev_seed_topic[s] = v["topic"]
            break

prev_bl_cats["topic"] = prev_bl_cats["seed"].map(prev_seed_topic)
prev_bl_cats = prev_bl_cats.drop(columns="max_level")

###-------------------------------

How do the two lists differ?

In [14]:
bl_cats_comb = pd.concat(
    [prev_bl_cats.assign(type="previous"), new_bl_cats.assign(type="new")],
    ignore_index=True,
)

In [15]:
bl_cats_comb["type"].value_counts()

type
previous    3385
new         3274
Name: count, dtype: int64

In [16]:
(
    bl_cats_comb
    .groupby(["type", "topic"])
    .agg(num_categories=pd.NamedAgg("name", "size"), max_level=pd.NamedAgg("level", "max"))
    .unstack(level=0)
)

num_categories          max_level         
type                          new previous       new previous
topic                                                        
child_abuse                   402      391         8        8
cruelty                       132      219         7        8
hateful_ideologies            303      298         4        4
pejoratives                    32       38         4        4
profanity                      11       11         2        2
sexually_explicit            2394     2428        10       10

Compute a diff between the two lists.

In [17]:
bl_diff = category.compare_categories_lists(prev_bl_cats, new_bl_cats)

Some categories have been added to or removed from topics, or else their details (seed/parent/level) have changed.
In some cases, a category may move from one topic to another, but remains on the blocklist.
These changes can be due to edits made to Wikipedia, or updates we made to our seed list.

In [18]:
bl_diff["diff"].value_counts().sort_index()

diff
added       34
changed     71
moved       42
removed    145
Name: count, dtype: int64

__NSFW content warning!__

List the categories that differ. Categories that have moved to a different topic are shown in blue.

In [19]:
display_pd(category.style_diff(bl_diff))

,name,seed_previous,parent_previous,level_previous,topic,seed_new,parent_new,level_new,diff
731,Canadian people convicted of child pornography offenses,Child abuse,Child pornography,3,child_abuse,Child abuse,People convicted of child pornography offenses,4,changed
734,Japanese people convicted of child pornography offenses,Child abuse,Child pornography,3,child_abuse,Child abuse,People convicted of child pornography offenses,4,changed
736,Singaporean people convicted of child pornography offenses,Child abuse,Child pornography,3,child_abuse,Child abuse,People convicted of child pornography offenses,4,changed
737,Welsh people convicted of child pornography offences,Child abuse,Child pornography,3,child_abuse,Child abuse,People convicted of child pornography offenses,4,changed
2343,Murdered Puerto Rican children,Child abuse,Murdered American children,5,child_abuse,--,--,--,removed
3395,Books about child prostitution,--,--,--,child_abuse,Child abuse,Works about child prostitution,4,moved
3398,Child prostitution in literature,--,--,--,child_abuse,Child abuse,Child sexual abuse in literature,4,moved
3403,Chinese people convicted of rape,--,--,--,child_abuse,Child abuse,Chinese rapists,6,moved
3399,Chinese rapists,--,--,--,child_abuse,Child abuse,Chinese sex offenders,5,moved
3396,Fictional child prostitutes,--,--,--,child_abuse,Child abuse,Works about child prostitution,4,moved


For the categories that have moved to a different topic, where did they end up?

In [20]:
category.style_diff(
    bl_diff.query("diff == 'moved'").sort_values(["name", "level_previous"])
)

,name,seed_previous,parent_previous,level_previous,topic,seed_new,parent_new,level_new,diff
549,Abu Ghraib torture and prisoner abuse,Sexual violence,Wartime sexual violence,2,sexually_explicit,--,--,--,moved
3386,Abu Ghraib torture and prisoner abuse,--,--,--,cruelty,Torture,Torture in Iraq,3,moved
125,BDSM equipment,Sex toys,Sex toys,1,sexually_explicit,--,--,--,moved
3385,BDSM equipment,--,--,--,cruelty,Torture,Instruments of torture,3,moved
1205,Books about child prostitution,Prostitution,Books about prostitution,3,sexually_explicit,--,--,--,moved
3395,Books about child prostitution,--,--,--,child_abuse,Child abuse,Works about child prostitution,4,moved
1211,Child prostitution in literature,Prostitution,Prostitution in literature,3,sexually_explicit,--,--,--,moved
3398,Child prostitution in literature,--,--,--,child_abuse,Child abuse,Child sexual abuse in literature,4,moved
2641,Chinese people convicted of rape,Sexual violence,People convicted of rape by nationality,5,sexually_explicit,--,--,--,moved
3403,Chinese people convicted of rape,--,--,--,child_abuse,Child abuse,Chinese rapists,6,moved
